In [43]:
# %load src/Graph.py
from collections import defaultdict
import random


class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = defaultdict(list)
        self.distances = {}

    def add_node(self, value):
        self.nodes.add(value)

    def add_nodes(self, list):
        for node in list:
            self.add_node(node)

    def add_edge(self, from_node, to_node, distance):
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.distances[(from_node, to_node)] = distance
        self.distances[(to_node, from_node)] = distance

    def add_edges(self, tuples):
        for t in tuples:
            self.add_edge(t[0], t[1], t[2])

    def remove_edge(self, edge):
        if self.distances.get(edge):
            self.edges[edge[0]].remove(edge[1])
            self.edges[edge[1]].remove(edge[0])
            self.distances.pop(edge)

    def remove_edges(self, edges):
        for edge in edges:
            self.remove_edge(edge)

    def remove_node(self, node):
        for n in self.nodes:
            if(node != n):
                self.remove_edge((node, n))
        self.nodes.remove(node)

    def remove_nodes(self, nodes):
        for node in nodes:
            self.remove_node(node)
    
    def random_graph(self, num_nodes, prob_axis):
        g = Graph()
        g.add_nodes(range(1, num_nodes + 1))
        for source in g.nodes:
            for target in g.nodes:
                if (source != target and random.random() < prob_axis and (not g.distances.get((source, target)))):
                    g.add_edge(source, target, random.randint(1, num_nodes))
        return g

    def dijkstra(self, initial):
        visited = {initial: 0}
        path = {}

        nodes = set(self.nodes)

        while nodes:
            min_node = None
            for node in nodes:
                if node in visited:
                    if min_node is None:
                        min_node = node
                    elif visited[node] < visited[min_node]:
                        min_node = node

            if min_node is None:
                break

            nodes.remove(min_node)
            current_weight = visited[min_node]

            for edge in self.edges[min_node]:
                weight = current_weight + self.distances[(min_node, edge)]
                if edge not in visited or weight < visited[edge]:
                    visited[edge] = weight
                    path[edge] = min_node

        return visited, path

    


In [46]:
graph = Graph()
g = graph.random_graph(10, .25)

print(g.distances)

(visited, path) = g.dijkstra(1)

print(visited)
print(path)

{(1, 10): 8, (10, 1): 8, (2, 1): 8, (1, 2): 8, (3, 6): 3, (6, 3): 3, (3, 9): 9, (9, 3): 9, (4, 3): 2, (3, 4): 2, (4, 6): 7, (6, 4): 7, (4, 10): 1, (10, 4): 1, (5, 1): 6, (1, 5): 6, (5, 7): 10, (7, 5): 10, (6, 7): 8, (7, 6): 8, (6, 9): 4, (9, 6): 4, (7, 1): 7, (1, 7): 7, (7, 4): 10, (4, 7): 10, (8, 4): 10, (4, 8): 10, (8, 5): 7, (5, 8): 7, (8, 6): 3, (6, 8): 3, (8, 9): 9, (9, 8): 9, (9, 1): 3, (1, 9): 3, (9, 2): 5, (2, 9): 5, (10, 3): 7, (3, 10): 7, (10, 7): 9, (7, 10): 9}
{1: 0, 10: 8, 2: 8, 5: 6, 7: 7, 9: 3, 3: 10, 6: 7, 8: 10, 4: 9}
{10: 1, 2: 1, 5: 1, 7: 1, 9: 1, 3: 6, 6: 9, 8: 6, 4: 10}
